In [1]:
# Install Google libraries
%pip install langchain-google-genai google-generativeai

In [ ]:
# Setup Dummy Database
import sqlite3
import pandas as pd

# Connect to SQLite database (creates it if it doesn't exist)
conn = sqlite3.connect('company_data.db')
cursor = conn.cursor()

# Create a sample table 'employees'
cursor.execute('''
CREATE TABLE IF NOT EXISTS employees (
    id INTEGER PRIMARY KEY,
    name TEXT,
    department TEXT,
    salary INTEGER,
    join_date TEXT
)
''')

# Insert dummy data
employees = [
    (1, 'Jason Ong', 'Engineering', 85000, '2023-01-15'),
    (2, 'Lim Li Kein', 'Data Science', 92000, '2022-11-20'),
    (3, 'Tang Yong Zhe', 'Marketing', 60000, '2024-02-10'),
    (4, 'Sarah Connor', 'Engineering', 88000, '2021-05-30'),
    (5, 'John Smith', 'HR', 55000, '2020-09-12')
]

cursor.executemany('INSERT OR IGNORE INTO employees VALUES (?,?,?,?,?)', employees)
conn.commit()

print("Database created and populated successfully!")

# Display the data to verify
df = pd.read_sql_query("SELECT * FROM employees", conn)
display(df) # Jupyter specific display command

In [4]:
# Model 1: Basic Model that Leverages Rule-Based or Pattern Identification with Dummy Dataset
import re

def rule_based_chatbot(user_query):
    query = user_query.lower()
    
    # Rule 1: Select All
    if "show all" in query and "employees" in query:
        sql = "SELECT * FROM employees"
        
    # Rule 2: Filter by Department (e.g., "Who works in Engineering?")
    elif "works in" in query:
        # Extract the last word as the department
        match = re.search(r'works in (\w+)', query)
        if match:
            dept = match.group(1)
            sql = f"SELECT * FROM employees WHERE department LIKE '{dept}%'"
        else:
            return "Error: Could not identify department."
            
    # Rule 3: Aggregation (Total/Average Salary)
    elif "average salary" in query:
        sql = "SELECT AVG(salary) FROM employees"
        
    else:
        return "I am a basic bot. I didn't understand that query."
    
    # Execute the generated SQL
    try:
        result_df = pd.read_sql_query(sql, conn)
        return result_df
    except Exception as e:
        return f"SQL Error: {e}"

# --- Testing Model 1 ---
print("Query 1: 'Show all employees'")
display(rule_based_chatbot("Show all employees"))

print("\nQuery 2: 'Who works in Engineering?'")
display(rule_based_chatbot("Who works in Engineering?"))

print("\nQuery 3: 'Show me the highest paid person'") 
# This will fail, illustrating the limitation mentioned in your Problem Statement 
print(rule_based_chatbot("Show me the highest paid person"))

Query 1: 'Show all employees'


,id,name,department,salary,join_date
0,1,Jason Ong,Engineering,85000,2023-01-15
1,2,Lim Li Kein,Data Science,92000,2022-11-20
2,3,Tang Yong Zhe,Marketing,60000,2024-02-10
3,4,Sarah Connor,Engineering,88000,2021-05-30
4,5,John Smith,HR,55000,2020-09-12



Query 2: 'Who works in Engineering?'


,id,name,department,salary,join_date
0,1,Jason Ong,Engineering,85000,2023-01-15
1,4,Sarah Connor,Engineering,88000,2021-05-30



Query 3: 'Show me the highest paid person'
I am a basic bot. I didn't understand that query.


In [5]:
# Fix 1 for Model 2
import langchain

# --- THE FIX ---
# Manually inject the missing attribute to stop the crash
if not hasattr(langchain, 'verbose'):
    langchain.verbose = False
# ----------------

import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.utilities import SQLDatabase
from langchain_community.agent_toolkits import create_sql_agent

# 1. Setup your Google Key (Make sure this is accurate)
os.environ["GOOGLE_API_KEY"] = "GOOGLE_API_KEY" # <--- PASTE YOUR KEY HERE

# 2. Connect to Database
db = SQLDatabase.from_uri("sqlite:///company_data.db")

# 3. Initialize Model
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0)

# 4. Create Agent
agent_executor = create_sql_agent(
    llm=llm,
    db=db,
    agent_type="zero-shot-react-description",
    verbose=True
)

# 5. Test
print("System fixed! Testing agent...")
try:
    response = agent_executor.invoke("Who is the highest paid employee?")
    print(f"AI Answer: {response['output']}")
except Exception as e:
    print(f"Error: {e}")

System fixed! Testing agent...
Error: module 'langchain' has no attribute 'debug'


In [6]:
# Fix 2 for Model 2
# 1. Upgrade all langchain libraries to sync their versions
%pip install -U langchain langchain-community langchain-core langchain-google-genai langchain-text-splitters

# 2. Install the missing dependency for the Google model if you haven't yet
%pip install -U google-generativeai

  Using cached langchain_core-1.1.0-py3-none-any.whl.metadata (3.6 kB)
  Using cached langchain_google_genai-3.2.0-py3-none-any.whl.metadata (2.7 kB)
  Using cached google_ai_generativelanguage-0.9.0-py3-none-any.whl.metadata (10 kB)
Using cached langchain_core-1.1.0-py3-none-any.whl (473 kB)
   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   ---------------------------------------- 1.4/1.4 MB 18.4 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.80
    Uninstalling langchain-core-0.3.80:
      Successfully uninstalled langchain-core-0.3.80
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.15
  Attempting uninstall: langchain-google-genai
    Found existing installation: langchain-google-genai 2.0.10
    Uninstalling lan

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.9.0 which is incompatible.


  Using cached google_ai_generativelanguage-0.6.15-py3-none-any.whl.metadata (5.7 kB)
Using cached google_ai_generativelanguage-0.6.15-py3-none-any.whl (1.3 MB)
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.9.0
    Uninstalling google-ai-generativelanguage-0.9.0:
      Successfully uninstalled google-ai-generativelanguage-0.9.0
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-google-genai 3.2.0 requires google-ai-generativelanguage<1.0.0,>=0.9.0, but you have google-ai-generativelanguage 0.6.15 which is incompatible.


In [2]:
# Fix 3 for Model 2
# Force upgrade the google-generativeai package to the latest version
%pip install --upgrade google-generativeai langchain-google-genai

INFO: pip is looking at multiple versions of langchain-google-genai to determine which version is compatible with other requirements. This could take a while.
  Using cached langchain_google_genai-3.2.0-py3-none-any.whl.metadata (2.7 kB)
  Using cached langchain_google_genai-3.1.0-py3-none-any.whl.metadata (2.7 kB)
  Using cached langchain_google_genai-3.0.3-py3-none-any.whl.metadata (2.7 kB)
  Using cached langchain_google_genai-3.0.2-py3-none-any.whl.metadata (2.7 kB)
INFO: pip is still looking at multiple versions of langchain-google-genai to determine which version is compatible with other requirements. This could take a while.
  Using cached langchain_google_genai-3.0.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached langchain_google_genai-3.0.0-py3-none-any.whl.metadata (7.1 kB)
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you 

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 1.1.0 requires langchain-core<2.0.0,>=1.1.0, but you have langchain-core 0.3.80 which is incompatible.
langchain-classic 1.0.0 requires langchain-core<2.0.0,>=1.0.0, but you have langchain-core 0.3.80 which is incompatible.
langchain-community 0.4.1 requires langchain-core<2.0.0,>=1.0.1, but you have langchain-core 0.3.80 which is incompatible.
langchain-openai 1.1.0 requires langchain-core<2.0.0,>=1.1.0, but you have langchain-core 0.3.80 which is incompatible.
langchain-text-splitters 1.0.0 requires langchain-core<2.0.0,>=1.0.0, but you have langchain-core 0.3.80 which is incompatible.
langgraph-prebuilt 1.0.5 requires langchain-core>=1.0.0, but you have langchain-core 0.3.80 which is incompatible.


In [9]:
# Fix 4 for Model 2
# Identify available models to be accessed 
import google.generativeai as genai
import os

# Ensure your key is set
os.environ["GOOGLE_API_KEY"] = "GOOGLE_API_KEY" # <--- PASTE YOUR KEY HERE

genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

print("Available Models:")
for m in genai.list_models():
    if 'generateContent' in m.supported_generation_methods:
        print(m.name)

Available Models:
models/gemini-2.5-pro-preview-03-25
models/gemini-2.5-flash
models/gemini-2.5-pro-preview-05-06
models/gemini-2.5-pro-preview-06-05
models/gemini-2.5-pro
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-flash-lite-preview
models/gemini-2.0-pro-exp
models/gemini-2.0-pro-exp-02-05
models/gemini-exp-1206
models/gemini-2.0-flash-thinking-exp-01-21
models/gemini-2.0-flash-thinking-exp
models/gemini-2.0-flash-thinking-exp-1219
models/gemini-2.5-flash-preview-tts
models/gemini-2.5-pro-preview-tts
models/learnlm-2.0-flash-experimental
models/gemma-3-1b-it
models/gemma-3-4b-it
models/gemma-3-12b-it
models/gemma-3-27b-it
models/gemma-3n-e4b-it
models/gemma-3n-e2b-it
models/gemini-flash-latest
models/gemini-flash-lite-latest
models/gemini-pro-latest
models/gemini-2.5-flash-lite
model

In [1]:
# Model 2: AI-Driven SQL Chatbot with Dummy Dataset
import langchain
import os
import sqlite3
import time
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.utilities import SQLDatabase
from langchain_community.agent_toolkits import create_sql_agent

# --- 1. THE COMPLETE PATCH ---
# Fixes 'verbose', 'debug', and 'llm_cache' errors
if not hasattr(langchain, 'verbose'):
    langchain.verbose = False
if not hasattr(langchain, 'debug'):
    langchain.debug = False
if not hasattr(langchain, 'llm_cache'):
    langchain.llm_cache = None

# --- 2. CONFIGURATION ---
# Replace with your actual key if needed
os.environ["GOOGLE_API_KEY"] = "GOOGLE_API_KEY" 

# --- 3. DATABASE CONNECTION ---
db_path = "company_data.db"
db = SQLDatabase.from_uri(f"sqlite:///{db_path}")

# --- 4. INITIALIZE MODEL ---
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0)

# --- 5. CREATE THE AGENT ---
agent_executor = create_sql_agent(
    llm=llm,
    db=db,
    agent_type="zero-shot-react-description",
    verbose=True, 
    handle_parsing_errors=True
)

# --- 6. DEFINE THE CHAT FUNCTION ---
def ask_database(question):
    print(f"\nUser Query: {question}")
    try:
        response = agent_executor.invoke(question)
        print(f"Final Answer: {response['output']}")
    except Exception as e:
        print(f"Error processing query: {e}")

# --- 7. TEST WITH DELAY ---

print("--- Query 1 ---")
ask_database("Who is the highest paid employee?")
time.sleep(10) # Wait 10 seconds to respect Google's free limit

print("\n--- Query 2 ---")
ask_database("What is the average salary of the Engineering department?")
time.sleep(10) # Wait 10 seconds to respect Google's free limit

print("\n--- Query 3 ---")
ask_database("List the names of employees who joined after the year 2022.")
time.sleep(10) # Wait 10 seconds to respect Google's free limit

# --- 8. GENERAL FORM ---
print("\n--- Query ---")
# Remove the # and enter your query into the quotation marks below 
#ask_database("Enter your prompt here")

--- Query 1 ---

User Query: Who is the highest paid employee?


> Entering new SQL Agent Executor chain...
Error processing query: Invalid argument provided to Gemini: 400 API key not valid. Please pass a valid API key. [reason: "API_KEY_INVALID"
domain: "googleapis.com"
metadata {
  key: "service"
  value: "generativelanguage.googleapis.com"
}
, locale: "en-US"
message: "API key not valid. Please pass a valid API key."
]

--- Query 2 ---

User Query: What is the average salary of the Engineering department?


> Entering new SQL Agent Executor chain...
Error processing query: Invalid argument provided to Gemini: 400 API key not valid. Please pass a valid API key. [reason: "API_KEY_INVALID"
domain: "googleapis.com"
metadata {
  key: "service"
  value: "generativelanguage.googleapis.com"
}
, locale: "en-US"
message: "API key not valid. Please pass a valid API key."
]

--- Query 3 ---

User Query: List the names of employees who joined after the year 2022.


> Entering new SQL Agent Exec

In [2]:
# Excel File Generation for Model 3
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta

# --- 1. GENERATE REALISTIC DATA ---
# We simulate a "Tech Store" database
products = {
    'Laptop': 1200, 'Smartphone': 800, 'Headphones': 150, 
    'Monitor': 300, 'Keyboard': 50, 'Mouse': 25, 'Smartwatch': 250
}
locations = ['New York', 'Los Angeles', 'Chicago', 'Houston', 'Phoenix']
categories = ['Computers', 'Mobile', 'Audio', 'Peripherals']

data = []
for i in range(1, 101): # Generate 100 records
    prod = random.choice(list(products.keys()))
    category = 'Computers' if prod in ['Laptop', 'Monitor'] else \
               'Mobile' if prod in ['Smartphone', 'Smartwatch'] else \
               'Audio' if prod == 'Headphones' else 'Peripherals'
    
    price = products[prod]
    qty = random.randint(1, 5)
    date = datetime(2023, 1, 1) + timedelta(days=random.randint(0, 600))
    
    data.append([
        1000 + i,                  # Order_ID
        date.strftime("%Y-%m-%d"), # Date
        prod,                      # Product
        category,                  # Category
        price,                     # Unit_Price
        qty,                       # Quantity
        price * qty,               # Total_Revenue
        random.choice(locations)   # City
    ])

# --- 2. SAVE TO EXCEL ---
df = pd.DataFrame(data, columns=[
    'Order_ID', 'Date', 'Product', 'Category', 
    'Unit_Price', 'Quantity', 'Total_Revenue', 'City'
])

file_name = "electronic_sales.xlsx"
df.to_excel(file_name, index=False)

print(f"✅ Success! Created '{file_name}' with {len(df)} rows.")
display(df.head()) # Show first 5 rows

✅ Success! Created 'electronic_sales.xlsx' with 100 rows.


,Order_ID,Date,Product,Category,Unit_Price,Quantity,Total_Revenue,City
0,1001,2023-03-15,Headphones,Audio,150,2,300,Chicago
1,1002,2023-06-18,Monitor,Computers,300,5,1500,Los Angeles
2,1003,2023-12-17,Laptop,Computers,1200,1,1200,Houston
3,1004,2023-09-27,Monitor,Computers,300,1,300,New York
4,1005,2024-05-13,Laptop,Computers,1200,1,1200,Chicago


In [2]:
# Model 3 (P1): AI-Driven SQL Chatbot with Excel File Dataset
# --- PART 1: SETUP & AUTOMATED TESTS (API KEY 1) ---
import langchain
import os
import sqlite3
import pandas as pd
import time
from langchain_community.utilities import SQLDatabase
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.agent_toolkits import create_sql_agent

# 1. SETUP LIBRARIES
if not hasattr(langchain, 'verbose'): langchain.verbose = False
if not hasattr(langchain, 'debug'): langchain.debug = False
if not hasattr(langchain, 'llm_cache'): langchain.llm_cache = None

# 2. LOAD DATABASE
excel_file = "electronic_sales.xlsx"
db_path = "retail_db.db"
try:
    df = pd.read_excel(excel_file)
    df.columns = [c.replace(' ', '_') for c in df.columns]
    conn = sqlite3.connect(db_path)
    df.to_sql('sales', conn, if_exists='replace', index=False)
    conn.close()
    print("Database built successfully.")
except Exception as e:
    print(f"Data Load Error: {e}")

# ==========================================
# 🛑 INPUT API KEY 1 HERE (For Tests)
# ==========================================
KEY_1 = "GOOGLE_API_KEY" 
os.environ["GOOGLE_API_KEY"] = KEY_1

# 3. INITIALIZE AGENT (With Key 1)
db = SQLDatabase.from_uri(f"sqlite:///{db_path}")
# Using 1.5-flash for stability
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0) 
agent_executor = create_sql_agent(
    llm=llm,
    db=db,
    agent_type="zero-shot-react-description",
    verbose=True,
    handle_parsing_errors=True
)

# 4. DEFINE FUNCTIONS
def ask_analytics(question):
    # Relies on the global 'agent_executor'
    response = agent_executor.invoke(question)
    print(f"\n🤖 Bot Answer: {response['output']}")

def ask_with_retry(question, delay=10):
    max_retries = 3
    for attempt in range(max_retries):
        try:
            print(f"\n--- Asking: {question} ---")
            ask_analytics(question)
            print(f"Success! Cooling down for {delay} seconds...")
            time.sleep(delay)
            return
        except Exception as e:
            if "429" in str(e) or "ResourceExhausted" in str(e):
                print(f"⚠️ Hit Rate Limit. Waiting 61s...")
                time.sleep(61)
            else:
                print(f"❌ Error: {e}")
                break

# 5. RUN AUTOMATED TESTS
print("="*60)
print("🚀 RUNNING TESTS WITH API KEY 1")
print("="*60)

ask_with_retry("What is the total revenue generated from Laptops?")
ask_with_retry("Which City has the highest number of orders?")
ask_with_retry("How many Smartphones were sold in 2024?")

Database built successfully.
🚀 RUNNING TESTS WITH API KEY 1

--- Asking: What is the total revenue generated from Laptops? ---


> Entering new SQL Agent Executor chain...
❌ Error: Invalid argument provided to Gemini: 400 API key not valid. Please pass a valid API key. [reason: "API_KEY_INVALID"
domain: "googleapis.com"
metadata {
  key: "service"
  value: "generativelanguage.googleapis.com"
}
, locale: "en-US"
message: "API key not valid. Please pass a valid API key."
]

--- Asking: Which City has the highest number of orders? ---


> Entering new SQL Agent Executor chain...
❌ Error: Invalid argument provided to Gemini: 400 API key not valid. Please pass a valid API key. [reason: "API_KEY_INVALID"
domain: "googleapis.com"
metadata {
  key: "service"
  value: "generativelanguage.googleapis.com"
}
, locale: "en-US"
message: "API key not valid. Please pass a valid API key."
]

--- Asking: How many Smartphones were sold in 2024? ---


> Entering new SQL Agent Executor chain...
❌ Error: In

In [3]:
# Model 3 (P2): AI-Driven SQL Chatbot with Excel File Dataset
# --- PART 2: INTERACTIVE SESSION (API KEY 2) ---

print("\n" + "="*60)
print("🔄 SWITCHING TO API KEY 2 FOR LIVE CHAT")
print("="*60)

# ==========================================
# 🛑 INPUT API KEY 2 HERE (For Chat)
# ==========================================
KEY_2 = "GOOGLE_API_KEY"
os.environ["GOOGLE_API_KEY"] = KEY_2

# 1. RE-INITIALIZE AGENT (With Key 2)
# We must recreate the LLM and Agent to pick up the new API Key
llm_chat = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0)

# Overwrite the global variable 'agent_executor'
agent_executor = create_sql_agent(
    llm=llm_chat,
    db=db,
    agent_type="zero-shot-react-description",
    verbose=True,
    handle_parsing_errors=True
)

# 2. START INTERACTIVE LOOP
print("💬 CHAT SESSION READY (Using Key 2)")
print("What would you like to ask?")
print("Type 'exit' to stop.")

while True:
    user_input = input("\n👤 You: ")
    
    if user_input.lower() in ["exit", "quit", "stop"]:
        print("Goodbye!")
        break
    
    if not user_input.strip():
        continue

    # Uses the UPDATED agent_executor automatically
    ask_with_retry(user_input, delay=10)


🔄 SWITCHING TO API KEY 2 FOR LIVE CHAT
💬 CHAT SESSION READY (Using Key 2)
What would you like to ask?
Type 'exit' to stop.



👤 You:  exit


Goodbye!
